In [1]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

In [2]:
# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)
n_steps = 2 if smoke_test else 200

# enable validation (e.g. validate parameters of distributions)
assert pyro.__version__.startswith('1.3.1')
pyro.enable_validation(True)

In [13]:
pyro.clear_param_store()

# data = []
# for _ in range(6):
#     data.append(torch.tensor(1.0))
# for _ in range(4):
#     data.append(torch.tensor(0.0))

data = torch.zeros(10)
data[0:6] = torch.ones(6)
data

tensor([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.])

In [ ]:
# Part1で使ったモデル
# def model(data):
#     alpha0 = torch.tensor(10.0)
#     beta0 = torch.tensor(10.0)
#     f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
#     for i in range(len(data)):
#         pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

In [ ]:
# plateを使ったversion（for文使用）
# def model(data):
#     alpha0 = torch.tensor(10.0)
#     beta0 = torch.tensor(10.0)
#     f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
#     for i in pyro.plate("data_loop", len(data)):
#         pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

In [ ]:
# plateのNGな使い方
def model(data):
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # WARNING do not do this!
    # my_reified_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    my_reified_list = list(pyro.plate("data_loop", len(data)))
    for i in my_reified_list:
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

In [18]:
# plateを使ったversion（for文なし）
def model(data):
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    with pyro.plate('observe_data'):
      pyro.sample('obs', dist.Bernoulli(f), obs=data)

In [19]:
def guide(data):
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0),
                         constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0),
                        constraint=constraints.positive)
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))

In [20]:
# setup the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

In [21]:
# do gradient steps
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')

....................

In [8]:
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

inferred_mean = alpha_q / (alpha_q + beta_q)
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nbased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))


based on the data and our prior belief, the fairness of the coin is 0.533 +- 0.090
